In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader,TensorDataset

from preprocess import *
from attacks import *
from models import *
from utils import *
from generate_attacks import *

device = 'cuda'
torch.cuda.set_device(2)

In [2]:
train_set = datasets.MNIST(root = '/home/aminul/data',train = True, 
                           transform = transforms.ToTensor(), download = False)
test_set = datasets.MNIST(root = '/home/aminul/data',train = False, 
                          transform = transforms.ToTensor(), download = False)

In [3]:
train_loader = DataLoader(train_set,100,True)
test_loader = DataLoader(test_set,100,False)

train_loader_2 = DataLoader(train_set,1,True)
test_loader_2 = DataLoader(test_set,1,False)

In [4]:
num_classes = 10
learning_rate = 0.01
num_epochs = 5

In [5]:
criterion = nn.CrossEntropyLoss()

model1 = CNN()

opt_1 = torch.optim.SGD(model1.parameters(),lr=learning_rate)

In [6]:
history = train(model1,train_loader,num_epochs,opt_1,criterion,device)
test(model1,test_loader,criterion,device)

  2%|▏         | 9/600 [00:00<00:07, 80.36it/s]

Training: epoch: [1/5]  loss: [2.09] Accuracy [40.56] 


  2%|▏         | 9/600 [00:00<00:07, 81.41it/s]

Training: epoch: [2/5]  loss: [0.52] Accuracy [85.02] 


  2%|▏         | 9/600 [00:00<00:07, 81.36it/s]

...

Training: epoch: [4/5]  loss: [0.25] Accuracy [92.42] 


  8%|▊         | 8/100 [00:00<00:01, 78.64it/s]

Training: epoch: [5/5]  loss: [0.20] Accuracy [93.97] 


100%|██████████| 100/100 [00:01<00:00, 80.83it/s]

Testing: Loss: [0.16] Accuracy [95.32]


In [7]:
adv_loader = generateFGSM(test_loader_2,model1,device,0.1,criterion)
test(model1,adv_loader,criterion,device)

  0% 25/10000 [00:00<00:41, 241.99it/s]

Generating Adversarial Images


100%|██████████| 100/100 [00:00<00:00, 568.40it/s]

Testing: Loss: [1.33] Accuracy [55.07]


In [8]:
eps= 0.1
momentum = 0.9
max_iter = 10
adv_loader = generateMIFGSM(test_loader_2,model1,device,eps,momentum,max_iter,criterion)
test(model1,adv_loader,criterion,device)

  0% 4/10000 [00:00<04:27, 37.39it/s]

Generating Adversarial Images


100%|██████████| 100/100 [00:00<00:00, 572.33it/s]

Testing: Loss: [6.34] Accuracy [0.97]


In [25]:
def deepfool(net, image, device, num_classes=10, overshoot=0.02, max_iter=10):
    
    net,image = net.to(device),image.to(device)
    
    f_image = net(image).cpu().data.numpy().flatten()
    I = (np.array(f_image)).flatten().argsort()[::-1]

    I = I[0:num_classes]
    label = I[0]

    input_shape = image.cpu().detach().numpy().shape
    pert_image = copy.deepcopy(image)
    w = np.zeros(input_shape)
    r_tot = np.zeros(input_shape)

    loop_i = 0

    x = torch.tensor(pert_image[None, :],requires_grad=True).to(device)
    
    fs = net.forward(x[0])
    fs_list = [fs[0,I[k]] for k in range(num_classes)]
    k_i = label

    while k_i == label and loop_i < max_iter:

        pert = np.inf
        fs[0, I[0]].backward(retain_graph=True)
        grad_orig = x.grad.data.cpu().numpy().copy()

        for k in range(1, num_classes):
            
            #x.zero_grad()
            
            fs[0, I[k]].backward(retain_graph=True)
            cur_grad = x.grad.data.cpu().numpy().copy()

            # set new w_k and new f_k
            w_k = cur_grad - grad_orig
            f_k = (fs[0, I[k]] - fs[0, I[0]]).data.cpu().numpy()

            pert_k = abs(f_k)/np.linalg.norm(w_k.flatten())

            # determine which w_k to use
            if pert_k < pert:
                pert = pert_k
                w = w_k

        # compute r_i and r_tot
        # Added 1e-4 for numerical stability
        r_i =  (pert+1e-4) * w / np.linalg.norm(w)
        r_tot = np.float32(r_tot + r_i)

        pert_image = image + (1+overshoot)*torch.from_numpy(r_tot).to(device)

        x = torch.tensor(pert_image, requires_grad=True).to(device)
        fs = net.forward(x[0])
        k_i = np.argmax(fs.data.cpu().numpy().flatten())

        loop_i += 1

    r_tot = (1+overshoot)*r_tot

    return r_tot, loop_i, label, k_i, pert_image

In [31]:
import copy

def generateDeepFool(test_loader,model,device,num_classes=10, overshoot=0.02, max_iter=10):
    adv_img = []
    l = []
     
    print("Generating Adversarial Images")
    iterator = tqdm(test_loader,ncols=0, leave=False)
    for data,labels in iterator:
        _,_,_,_,x1 = deepfool(model,data,device,num_classes=10, overshoot=0.02, max_iter=10)
        l.append(labels)
        x2 = x1.squeeze().cpu().detach().numpy()
        adv_img.append(x2)
            
    x3 = np.array(adv_img)
    x3 = x3.reshape(10000,1,28,28)
    l = np.array(l)
        
    features_test = torch.from_numpy(x3)
    targets_test = torch.from_numpy(l)

    new_dataset = torch.utils.data.TensorDataset(features_test,targets_test)
    new_data_loader = torch.utils.data.DataLoader(new_dataset,100,shuffle = False)
    
    
    return new_data_loader

In [ ]:
adv_loader = generateDeepFool(test_loader_2,model1,device,num_classes=10, overshoot=0.02, max_iter=10)
test(model1,adv_loader,criterion,device)

  0% 0/10000 [00:00<?, ?it/s]

Generating Adversarial Images


/data/aminul/anaconda3/envs/pyTorch/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/data/aminul/anaconda3/envs/pyTorch/lib/python3.6/site-packages/ipykernel_launcher.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  4% 355/10000 [00:37<18:56,  8.49it/s]